In [28]:
%reload_ext autoreload
%autoreload 2

In [29]:
import bpy
import sys
import tempfile
import typst
from pathlib import Path

project_root = Path.home() / "projects/blender_typst_importer/"
sys.path.append(str(project_root))


In [33]:
from typst_importer.typst_to_svg import typst_express
from typst_importer.notebook_utils import display_svg

def eq(equation: str, name: str, color: str, position=(0, 0, 0), show_indices=False):

    typst_code = f"""
    #set text({color})
    $ {equation} $
    """
    collection = typst_express(
        typst_code,
        origin_to_char=True,
        convert_to_mesh=True,
        name=name,
        position=position,
        show_indices=show_indices
    )
    display_svg(collection.processed_svg, width="230px")
    return collection
imported_collection = eq("a = b + c", "Start", "aqua", (0, 0, 0.1), show_indices=False)
# eq("a = b + c", "End", "olive", (0, -0.8, 0));

In [34]:

# Create a new collection for indices if there are multiple objects
if len(imported_collection.objects) > 1:
    indices_collection = bpy.data.collections.new(f"{imported_collection.name}_Indices")
    bpy.context.scene.collection.children.link(indices_collection)
    
    for i, obj in enumerate(imported_collection.objects):
        # Create text object at the same location as the curve/mesh
        bpy.ops.object.text_add(location=obj.location)
        text_obj = bpy.context.active_object
        text_obj.data.body = str(i)
        text_obj.name = f"Index_{i}"
        
        # Set text color to red
        if "Index_Material" not in bpy.data.materials:
            mat = bpy.data.materials.new("Index_Material")
            mat.diffuse_color = (1.0, 0.0, 0.0, 1.0)  # Red color
        else:
            mat = bpy.data.materials["Index_Material"]
        
        if text_obj.data.materials:
            text_obj.data.materials[0] = mat
        else:
            text_obj.data.materials.append(mat)
        
        # Move from scene collection to indices collection
        bpy.context.scene.collection.objects.unlink(text_obj)
        indices_collection.objects.link(text_obj)
